### Youtube Sentence Pos Tag Analysis

In [16]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [17]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [18]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# data file parameter
file = "200 Classification"  # must be excel file
sheets = "Sheet3"  # "Sheet3"
start_first_column = 0  # start column location number

# classification
threshold_num = 1

In [19]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [20]:
def column_word_list(df, column):
    word_list = []
    for i in df[f"{column}"].dropna():
        words = i.split(",")
        for j in words:
            word = j.strip()
            word_list.append(word)
    return word_list

In [21]:
def column_word(df, column_list):
    '''
    column_word(df_pos_tag_group, df_pos_tag_group.columns) \n
    column_word(df_pos_tag_group, ["col_1","col_2"])
    '''
    word_list_all = []
    for i in column_list:
        word_list = column_word_list(df, f"{i}")
        word_list_all.append(word_list)
    return word_list_all

In [22]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    if any([True if i>count_condition else False for i in count_list]):
        return "Fail"
    else:
        return "Pass"

    #return any([True if i>count_condition else False for i in count_list])  

In [23]:
df_pos_tag_group = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/\
Data/{lang_folder.capitalize()}/{file}.xlsx", sheet_name=f"{sheets}")
df_pos_tag_group

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8
0,adam,başka,al,ama,"ben, beni, benimle, bana, bence, benim",artık,aynı,dostum,"mı, mısın"
1,anne,belki,"bak, bakalım",çok,"bir, biri",asla,biraz,efendim,"misin, mi"
2,baba,bile,"biliyorum, biliyorsun, bilmiyorum, biliyor",da,"bizi, bizim, bize, biz",birkaç,büyük,hadi,mü
3,"gerçek, gerçekten",birlikte,"değil, değilim",daha,"bu, bunun, buna, bunu, burada, buraya",bugün,doğru,haydi,"mu, musun"
4,iş,böyle,"demek, de",diye,"o, onu, onları, ona, onun, onlar, onunla, orada",dakika,en,hey,nasıl
5,izin,bütün,dur,evet,"seni, seninle, senin, sen, sana",devam,güzel,"iş, işte","ne, neler"
6,kız,çünkü,"ederim, et",gibi,"size, sizi, siz",gece,harika,işte,neden
7,önemli,eğer,"gel, geldi, geliyor",hayır,"şu, şunu",gün,iki,lütfen,nerede
8,özür,fazla,git,için,NaN,hala,iyi,merhaba,NaN
9,para,gerek,ister,ile,NaN,hemen,kötü,oh,NaN


In [24]:
df_youtube_sent_select = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/\
2-Word Group In Youtube Sentence/Turkish English/Turkish_English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN


In [25]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [26]:
result_list = []
#for i in df_youtube_sent_select["search_string"]:
for i in range(len(df_youtube_sent_select)):
    #words = word_tokenize(i)
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    words = word_tokenize(search)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([search,start,end,sent,id,id_url,list_var])

In [27]:
df_youtube_sent_class = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","classification"])
df_youtube_sent_class

,search_string,start_time,end_time,sentence,video_id,video_url,classification
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[0, 2, 3, 7, 3, 5, 1, 5, 7, 1, 5, 5]"
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[2, 1, 1, 1, 2, 1, 3, 1, 2, 3, 1, 2, 8, 2]"
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[7, 1, 4, 1, 2, 8, 7, 3, 4, 1, 2, 8, 7, 8, 8]"
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[4, 3, 0, 2, 6, 1, 5, 4, 2, 4, 0, 5]"
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...,"[2, 2, 4, 3, 2, 2, 4, 3, 2, 3, 5, 0, 7, 4, 3]"
...,...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN,"[4, 8, 4, 8]"
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN,"[4, 8, 3, 4]"
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN,"[4, 8, 8, 4]"
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN,"[4, 8, 4, 8]"


In [28]:
df_youtube_sent_class["condition"] = df_youtube_sent_class["classification"].apply(lambda x: classification_condition(x, threshold_num))
df_youtube_sent_class

,search_string,start_time,end_time,sentence,video_id,video_url,classification,condition
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[0, 2, 3, 7, 3, 5, 1, 5, 7, 1, 5, 5]",Fail
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[2, 1, 1, 1, 2, 1, 3, 1, 2, 3, 1, 2, 8, 2]",Fail
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[7, 1, 4, 1, 2, 8, 7, 3, 4, 1, 2, 8, 7, 8, 8]",Fail
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[4, 3, 0, 2, 6, 1, 5, 4, 2, 4, 0, 5]",Fail
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...,"[2, 2, 4, 3, 2, 2, 4, 3, 2, 3, 5, 0, 7, 4, 3]",Fail
...,...,...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN,"[4, 8, 4, 8]",Fail
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN,"[4, 8, 3, 4]",Fail
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN,"[4, 8, 8, 4]",Fail
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN,"[4, 8, 4, 8]",Fail


In [29]:
df_youtube_sent_class.to_excel(f"{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_{threshold_num}_Classification_All.xlsx", index=False)

In [30]:
df_youtube_sent_class_select = df_youtube_sent_class[df_youtube_sent_class["condition"] == "Pass"]
df_youtube_sent_class_select.reset_index(drop=True, inplace=True)
df_youtube_sent_class_select

,search_string,start_time,end_time,sentence,video_id,video_url,classification,condition
0,ya gerçekten hiç gerek yok ben çok iyi,2146,2148,ya gerçekten hiç gerek yok ben çok iyi bakacağ...,qV3xntSmr1o,https://www.youtube.com/watch?v=qV3xntSmr1o&t=...,"[7, 0, 5, 1, 2, 4, 3, 6]",Pass
1,mu daha önce seninle birlikte olduğunu,158,160,bahar bilmiyor mu daha önce seninle birlikte o...,qVk4p_4pwv8,https://www.youtube.com/watch?v=qVk4p_4pwv8&t=...,"[8, 3, 5, 4, 1, 2]",Pass
2,çünkü burada neler olduğunu gerçekten,1450,1453,çünkü burada neler olduğunu gerçekten bilmek i...,OmvrQaXfrZQ,https://www.youtube.com/watch?v=OmvrQaXfrZQ&t=...,"[1, 4, 8, 2, 0]",Pass
3,gerçekten çok teşekkür ederim o zaman,391,396,meral hanım gerçekten çok teşekkür ederim o zaman,fLgorIU0es4,https://www.youtube.com/watch?v=fLgorIU0es4&t=...,"[0, 3, 7, 2, 4, 5]",Pass
4,geliyor adam gibi bir şeyler şimdi ne,1192,1197,iyi olsa bile getirmiyorsun ne yapacağız bugün...,JyzAOR0GTs4,https://www.youtube.com/watch?v=JyzAOR0GTs4&t=...,"[2, 0, 3, 4, 1, 5, 8]",Pass
...,...,...,...,...,...,...,...,...
32260,ya da hiç o,4949,4950,ya da hiç o riske girme direkt git buradan ya,dHohXDBK3KI,NaN,"[7, 3, 5, 4]",Pass
32261,o kız mı da,142,145,o kız mı da belirliyorum,oDt7oJ_raio,NaN,"[4, 0, 8, 3]",Pass
32262,var o da ne,670,673,burası çünkü bir dükkan canım ya kaşgarlı türk...,y4N3us_VXRA,NaN,"[2, 4, 3, 8]",Pass
32263,o ne ki ya,3512,3513,o ne ki ya kısa oruç öğlene kadar,KHyLRPII8ow,NaN,"[4, 8, 3, 7]",Pass


In [31]:
df_youtube_sent_class_select.to_excel(f"{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_{threshold_num}_Classification_Select.xlsx", index=False)

#### Copy Move And Delete

In [32]:
output_file = glob.glob(f"{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_*_Classification_*.xlsx")
output_file

['Turkish_200_Word_Group_In_Youtube_Sentence_1_Classification_All.xlsx',
 'Turkish_200_Word_Group_In_Youtube_Sentence_1_Classification_Select.xlsx']

In [33]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [34]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

In [88]:
def classification_order(classification_list, target_pattern_list):
    '''
    classification_order(classification_list, target_pattern_list)
    classification_list and target_pattern_list are list
    '''
    if [x for x in classification_list] == [y for y in target_pattern_list]:
        return "pattern is equal"
    else:
        return "pattern in not equal"

In [89]:
target_pattern = [7,1,4,1,2,8,7,3,4,1,2,8,7,8,8]

In [90]:
df_youtube_sent_class["test"] = df_youtube_sent_class["classification"].apply(lambda x: classification_order(x, target_pattern))

In [91]:
df_youtube_sent_class

,search_string,start_time,end_time,sentence,video_id,video_url,classification,condition,test
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[0, 2, 3, 7, 3, 5, 1, 5, 7, 1, 5, 5]",Fail,pattern in not equal
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[2, 1, 1, 1, 2, 1, 3, 1, 2, 3, 1, 2, 8, 2]",Fail,pattern in not equal
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[7, 1, 4, 1, 2, 8, 7, 3, 4, 1, 2, 8, 7, 8, 8]",Fail,pattern is equal
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[4, 3, 0, 2, 6, 1, 5, 4, 2, 4, 0, 5]",Fail,pattern in not equal
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...,"[2, 2, 4, 3, 2, 2, 4, 3, 2, 3, 5, 0, 7, 4, 3]",Fail,pattern in not equal
...,...,...,...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN,"[4, 8, 4, 8]",Fail,pattern in not equal
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN,"[4, 8, 3, 4]",Fail,pattern in not equal
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN,"[4, 8, 8, 4]",Fail,pattern in not equal
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN,"[4, 8, 4, 8]",Fail,pattern in not equal


In [92]:
df_youtube_sent_class[df_youtube_sent_class["test"] == "pattern is equal"]

,search_string,start_time,end_time,sentence,video_id,video_url,classification,condition,test
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[7, 1, 4, 1, 2, 8, 7, 3, 4, 1, 2, 8, 7, 8, 8]",Fail,pattern is equal


In [87]:
[0,1,5] == [0,1,6]

False

In [ ]:
column_word_list(df_pos_tag_group, "col_1")

In [ ]:
column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
df_test = df_youtube_sent_select.head(2)
df_test

In [ ]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [ ]:
result_list = []
for i in df_test["search_string"]:
    words = word_tokenize(i)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([i,list_var])

In [ ]:
result_list

In [ ]:
pd.DataFrame(result_list, columns=["sentence","classification"])

In [ ]:
len(column_word_list_all)

In [ ]:
from collections import Counter

In [ ]:
result_list[0][1]

In [ ]:
Counter(result_list[0][1])

In [ ]:
any(Counter(result_list[0][1]).values())

In [ ]:
list_test = list(Counter(result_list[0][1]).values())
list_test

In [ ]:
any([True if i>3 else False for i in list_test])

In [ ]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    return any([True if i>count_condition else False for i in count_list])   

In [ ]:
list_test2 = result_list[0][1]

In [ ]:
classification_condition(list_test2, 4)